In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import tqdm
import torch.utils.data as data
import matplotlib.pyplot as plt

In [ ]:
timeseries = 

train_size = 
test_size = 

train, test = timeseries[:train_size], timeseries[-test_size:]

def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset
    
    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    values = dataset.values
    X, y = [], []
    for i in range(len(values)-lookback):
        feature = values[i:i+lookback].reshape(-1, 1)
        target = values[i+1:i+lookback+1].reshape(-1, 1)
        X.append(feature)
        y.append(target)
    return torch.tensor(X), torch.tensor(y)

lookback = 7 
X_train, y_train = create_dataset(train, lookback=lookback)
X_test, y_test = create_dataset(test, lookback=lookback)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x   

In [ ]:
model = LSTM()
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.MSELoss()
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=8)

n_epochs = 200
train_errors = []
test_errors = []

for epoch in tqdm(range(n_epochs)):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Validation
    if epoch % 10 == 0:  
        model.eval()
        with torch.no_grad():
            y_pred_train = model(X_train)
            train_rmse = np.sqrt(loss_fn(y_pred_train, y_train))
            y_pred_test = model(X_test)
            test_rmse = np.sqrt(loss_fn(y_pred_test, y_test))
        
        train_errors.append(train_rmse.item())
        test_errors.append(test_rmse.item())

        print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

# Plotting the training and validation errors
plt.plot(range(0, n_epochs, 10), train_errors, label='Train RMSE')
plt.plot(range(0, n_epochs, 10), test_errors, label='Test RMSE')
plt.xlabel('Epoch')
plt.ylabel('RMSE')
plt.legend()